#Surivor - Outwit, Outplay, Outlast

## Imports and functions

In [2]:
%pip install gender_guesser
%pip install pymysql

from bs4 import BeautifulSoup

import requests
import urllib.request
import time
import random
import requests
import random
import os
import pymysql
import time
import gender_guesser.detector as gender
import pandas as pd
import numpy as np

STATES = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'D.C.': 'DC', 'Florida': 'FL', 
          'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 
          'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 
          'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 
          'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 
          'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA.', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


SEASONS = ['Borneo', 'The_Australian_Outback', 'Africa', 'Marquesas', 'Thailand', 'The_Amazon', 'Pearl_Islands', 'All-Stars', 'Vanuatu'] 

def getSeasonURL(season_num):
  wiki_url = 'https://en.wikipedia.org/wiki/Survivor:_'
  return wiki_url + SEASONS[ season_num - 1 ]

def getSurvivorId(first_name, last_name):
  first_name = "'"+first_name+"'"
  last_name = "'"+last_name.strip(" ")+"'"
  query = "Select survivor_id from survivor_data.Players where first_name ="+first_name+" and last_name="+last_name+";"
  print(query)
  cur.execute(query)
  id = cur.fetchone()[0]
  return str(id)

def executeQuery(query):
  cur.execute(query)

def commitQuery():
  conn.commit()

def findStateInString(s):
  word = ''
  for char in s:
    if word in states:
      return word
    else:
      word += char

def addQuotes(s):
  return " '"+s+"' "

def getGender(name):
  d = gender.Detector()
  g = d.get_gender(name).split("_")

  if len(g) == 2:
    gn = g[-1]
  else:
    gn = g[0]
    
  if gn == "male":
    return 'M'
  else:
    return 'F'

def getPlayerTableInfoRandom(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table', {'class': 'wikitable'})[0]
  table_rows = table.find_all('tr')

  rows = []
  for tr in table_rows:
      td = tr.find_all('td')
      td_row = [i.text for i in td]
      rows.append(td_row[:-1])
      random.shuffle(rows)

  return rows

def getPlayerTableInfo(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table', {'class': 'wikitable'})[0]
  table_rows = table.find_all('tr')

  rows = []
  for tr in table_rows:
      td = tr.find_all('td')
      td_row = [i.text for i in td]
      rows.append(td_row[:-1])

  return rows

     |████████████████████████████████| 379 kB 32.0 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 


## AWS Login

In [ ]:
  succeess = False

  while not succeess:
    password = input("Password: ");
    try:
      conn = pymysql.connect(
          host='aws-survivor.cpapv35x7rsh.us-east-2.rds.amazonaws.com',
          port=int(3306),
          user="admin",
          passwd=password,
          db="survivor_data",
          charset='utf8mb4')
      cur = conn.cursor()
      succeess = True
    except:
      print("Wrong Password")
  print("Connection Made")

Password: Frederickwilliams121234
Connection Made


In [ ]:
cur.close()

## Adding Players to Players Table

In [ ]:
url = SEASONS[8]
rows =getPlayerTableInfoRandom(url)
print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0]
      last_name  = name_age[:-2].split(" ", 1)[1]
      age = name_age[-2:]
      gen = getGender(first_name)
      state = r_split[-1].strip("\n")[1:]

      print(first_name, last_name, age, gen, state)
      try:
        query = "INSERT INTO `survivor_data`.`Players` (`first_name`, `last_name`, `sex`, `state`) VALUES ("+addQuotes(first_name)+","+addQuotes(last_name)+","+addQuotes(gen)+","+addQuotes(STATES[state])+");"
        #executeQuery(query)
      except:
        print("Error")

#commitQuery()

[['Twila Tanner41, Marshall, Missouri\n', 'Yasur\n', 'Lopevi\n'], ['Travis "Bubba" Sampson33, Johnson City, Tennessee\n', 'Lopevi\n', 'Yasur\n'], ['Brady Finta33, Huntington Beach, California\n', 'Lopevi\n'], ['Ami Cusack31, Lakewood, Colorado\n', 'Yasur\n', 'Yasur\n'], ['Lisa Keiffer44, Mandeville, Louisiana\n', 'Yasur\n', 'Yasur\n'], ['Rory Freeman35, Des Moines, Iowa\n', 'Lopevi\n', 'Yasur\n', 'Alinta\n'], ['Leann Slaby35, Kansasville, Wisconsin\n', 'Yasur\n', 'Yasur\n'], ['Brook Geraghty27, Winthrop, Massachusetts\n', 'Lopevi\n', '\n', '\n'], ['Scout Cloud Lee59, Stillwater, Oklahoma\n', 'Yasur\n', 'Yasur\n'], ['Chris Daugherty33, South Vienna, Ohio\n', 'Lopevi\n', 'Lopevi\n'], ['Julie Berry23, Gorham, Maine\n', 'Yasur\n', 'Lopevi\n'], ['Lea "Sarge" Masters40, Columbia, South Carolina\n', 'Lopevi\n', 'Lopevi\n'], ['Mia Galeotalanza29, Boston, Massachusetts\n', 'Yasur\n'], ['John Kenney22, Los Angeles, California\n', 'Lopevi\n', 'Lopevi\n'], ['James "Chad" Crittenden34, Oakland, Cal

## Add Players to Seasons Table

In [ ]:
season = 9
url = SEASONS[season - 1]
rows =getPlayerTableInfoRandom(url)
print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0]
      last_name  = name_age[:-2].split(" ", 1)[1]
      age = name_age[-2:]

      try:
        survivor_id = getSurvivorId(first_name, last_name)
        query = "INSERT INTO `survivor_data`.`Seasons` (season_id, survivor_id, age)" +" VALUES('"+str(season)+"','"+survivor_id+"','"+str(age)+"');"
        print(query)
        #executeQuery(query)
      except:
        print(first_name, last_name, age)

#commitQuery()


[['Travis "Bubba" Sampson33, Johnson City, Tennessee\n', 'Lopevi\n', 'Yasur\n'], ['Leann Slaby35, Kansasville, Wisconsin\n', 'Yasur\n', 'Yasur\n'], ['Scout Cloud Lee59, Stillwater, Oklahoma\n', 'Yasur\n', 'Yasur\n'], ['Rory Freeman35, Des Moines, Iowa\n', 'Lopevi\n', 'Yasur\n', 'Alinta\n'], ['Julie Berry23, Gorham, Maine\n', 'Yasur\n', 'Lopevi\n'], ['Dolly Neely25, Lake Latonka, Pennsylvania\n', 'Yasur\n'], ['John Kenney22, Los Angeles, California\n', 'Lopevi\n', 'Lopevi\n'], ['Mia Galeotalanza29, Boston, Massachusetts\n', 'Yasur\n'], ['Eliza Orlins21, Syracuse, New York\n', 'Yasur\n', 'Yasur\n'], ['Twila Tanner41, Marshall, Missouri\n', 'Yasur\n', 'Lopevi\n'], ['Brook Geraghty27, Winthrop, Massachusetts\n', 'Lopevi\n', '\n', '\n'], ['James "Chad" Crittenden34, Oakland, California\n', 'Lopevi\n', 'Lopevi\n'], [], ['John Palyok31, Los Angeles, California\n', 'Lopevi\n'], ['Brady Finta33, Huntington Beach, California\n', 'Lopevi\n'], ['Lisa Keiffer44, Mandeville, Louisiana\n', 'Yasur\n',

## Add Votes for in Season Table

In [ ]:
a = [ ['Lea', 'Masters'], ['Lisa', 'Keiffer'], ['John', 'Palyok'] ]
v = [ ['Brady', 'Finta'], ['Julie', 'Berry'], ['Chris', 'Daugherty'] ]
season = 9


for player in a:
  survivor_id = getSurvivorId(first_name=player[0], last_name=player[1])
  query = "UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='"+str(season)+"' and survivor_id ='"+survivor_id+"';"
  executeQuery(query)
  print(query)

for player in v:
  survivor_id = getSurvivorId(first_name=player[0], last_name=player[1])
  query = "UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='"+str(season)+"' and `survivor_id ='"+survivor_id+"';"
  executeQuery(query)
  print(query)

commitQuery()

Select survivor_id from survivor_data.Players where first_name ='Lea' and last_name='Masters';
UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='9' and survivor_id ='125';
Select survivor_id from survivor_data.Players where first_name ='Lisa' and last_name='Keiffer';
UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='9' and survivor_id ='133';
Select survivor_id from survivor_data.Players where first_name ='John' and last_name='Palyok';
UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='9' and survivor_id ='134';
Select survivor_id from survivor_data.Players where first_name ='Brady' and last_name='Finta';
UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='9' and survivor_id ='129';
Select survivor_id from survivor_data.Players where first_name ='Julie' and last_name='Berry';
UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='9' and survivor_id ='121';
Select survivor_

## Set Players Place in the Season

In [ ]:
season = 8
url = SEASONS[season - 1]
rows =getPlayerTableInfo(url)
rows = rows[1:]
place = len(rows)

print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0].strip(" ")
      last_name  = name_age[:-2].split(" ", 1)[1].strip(" ").strip("'")
      #survivor_id = getSurvivorId(first_name, last_name)

      try:
        survivor_id = getSurvivorId(first_name, last_name)
        query = "UPDATE `survivor_data`.`Seasons` SET place ="+ str(place) +" WHERE `season_id` = 8 and `survivor_id` ='"+survivor_id+"';"
        #executeQuery(query)

      except:
        print("Error: ", first_name, last_name) 
        
  place -= 1   

#commitQuery()

## Getting Challenge Winners

In [10]:
response = requests.get(getSeasonURL(7))
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table', {'class': 'wikitable'})[1]
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    td = tr.find_all('td')
    td_row = [i.text for i in td]
    if len(td_row) != 0:
      rows.append(td_row)

print(rows)

count = 0

for epi in rows:
  if count >= 8:
    reward = 'N/A'
    immunity = 'N/A'

    print(epi)

    if epi[-1].strip("\n") != 'Runner-up' or epi[-1].strip("\n") != 'Sole Survivor':
      if epi[-1][:2] == '13' or epi[-1][:2] == '14':
        immunity = epi[-3].strip("\n")
      elif len(epi) >= 4:
        reward = epi[-4].strip("\n")
        immunity = epi[-3].strip("\n")
    
    print(reward, immunity)
  count +=1

[['"Beg, Barter, Steal"\n', 'September 18, 2003\n', 'None\n', 'Drake\n', 'Nicole\n', '1st voted outDay 3\n'], ['"To Quit or Not to Quit"\n', 'September 25, 2003\n', 'Drake\n', 'Drake\n', 'Ryan\xa0S.\n', '2nd voted outDay 6\n'], ['"United We Stand, Divided We...?"\n', 'October 2, 2003\n', 'Drake\n', 'Drake\n', 'Lillian\n', '3rd voted outDay 9\n'], ['"Pick a Castaway...Any Castaway"\n', 'October 9, 2003\n', 'Drake\n', 'Morgan\n', 'Burton\n', '4th voted outDay 12\n'], ['[Rupert][a]\n'], ['"Everyone\'s Hero"\n', 'October 16, 2003\n', 'Morgan\n', 'Morgan\n', 'Michelle\n', '5th voted outDay 15\n'], ['"Me and My Snake"\n', 'October 23, 2003\n', 'Drake\n', 'Morgan\n', 'Trish\n', '6th voted outDay 18\n'], ['"What the...?, Part 1"\n', 'October 30, 2003\n', 'None\n', 'Outcasts[b]\n', 'Shawn\n', '7th voted outDay 19\n'], ['Osten\n', 'QuitDay 19\n'], ['"What the...?, Part 2"\n', 'November 6, 2003\n', 'None[c]\n'], ['Burton[d]\n', 'Andrew\n', '8th voted outDay 21\n'], ['Lillian[d]\n'], ['Burton (Rup